In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('train_ctrUa4K.csv')
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [3]:
train['Gender']= train['Gender'].map({'Male':0, 'Female':1})
train['Married']= train['Married'].map({'No':0, 'Yes':1})
train['Loan_Status']= train['Loan_Status'].map({'N':0, 'Y':1})

In [4]:
train.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [5]:
train = train.dropna()

In [6]:
train.shape

(480, 13)

In [7]:
train.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [8]:
train.describe()

,Gender,Married,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status
count,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000
mean,0.179167,0.647917,5364.231250,1581.093583,144.735417,342.050000,0.854167,0.691667
std,0.383892,0.478118,5668.251251,2617.692267,80.508164,65.212401,0.353307,0.462287
min,0.000000,0.000000,150.000000,0.000000,9.000000,36.000000,0.000000,0.000000
25%,0.000000,0.000000,2898.750000,0.000000,100.000000,360.000000,1.000000,0.000000
50%,0.000000,1.000000,3859.000000,1084.500000,128.000000,360.000000,1.000000,1.000000
75%,0.000000,1.000000,5852.500000,2253.250000,170.000000,360.000000,1.000000,1.000000
max,1.000000,1.000000,81000.000000,33837.000000,600.000000,480.000000,1.000000,1.000000


In [9]:
X = train[['Gender', 'Married', 'ApplicantIncome', 'LoanAmount','Credit_History']]
y = train.Loan_Status
X.shape, y.shape

((480, 5), (480,))

In [10]:
from sklearn.model_selection import train_test_split
x_train, x_cv, y_train, y_cv = train_test_split(X,y, test_size = 0.2,
random_state = 10)

In [11]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth=4, random_state = 10)
model.fit(x_train, y_train)

RandomForestClassifier(max_depth=4, random_state=10)

In [12]:
from sklearn.metrics import accuracy_score
pred_cv = model.predict(x_cv)
accuracy_score(y_cv,pred_cv)

0.8020833333333334

In [13]:
pred_train = model.predict(x_train)
accuracy_score(y_train,pred_train)

0.8203125

In [14]:
# saving the model
import pickle
pickle_out = open("classifier.pkl", mode = "wb")
pickle.dump(model, pickle_out)
pickle_out.close()

In [15]:
!pip install -q pyngrok

In [16]:
!pip install -q streamlit

In [17]:
!pip install -q streamlit_ace

In [18]:
import pickle
import streamlit as st
 
# loading the trained model
pickle_in = open('classifier.pkl', 'rb') 
classifier = pickle.load(pickle_in)
 
@st.cache()
  
# defining the function which will make the prediction using the data which the user inputs 
def prediction(Gender, Married, ApplicantIncome, LoanAmount, Credit_History):   
 
    # Pre-processing user input    
    if Gender == "Male":
        Gender = 0
    else:
        Gender = 1
 
    if Married == "Unmarried":
        Married = 0
    else:
        Married = 1
 
    if Credit_History == "Unclear Debts":
        Credit_History = 0
    else:
        Credit_History = 1  
 
    LoanAmount = LoanAmount / 1000
 
    # Making predictions 
    prediction = classifier.predict( 
        [[Gender, Married, ApplicantIncome, LoanAmount, Credit_History]])
     
    if prediction == 0:
        pred = 'Rejected'
    else:
        pred = 'Approved'
    return pred
      
# this is the main function in which we define our webpage  
def main():       
    # front end elements of the web page 
    html_temp = """ 
    <div style ="background-color:yellow;padding:13px"> 
    <h1 style ="color:black;text-align:center;">Streamlit Loan Prediction ML App</h1> 
    </div> 
    """
      
    # display the front end aspect
    st.markdown(html_temp, unsafe_allow_html = True) 
      
    # following lines create boxes in which user can enter data required to make prediction 
    Gender = st.selectbox('Gender',("Male","Female"))
    Married = st.selectbox('Marital Status',("Unmarried","Married")) 
    ApplicantIncome = st.number_input("Applicants monthly income") 
    LoanAmount = st.number_input("Total loan amount")
    Credit_History = st.selectbox('Credit_History',("Unclear Debts","No Unclear Debts"))
    result =""
      
    # when 'Predict' is clicked, make the prediction and store it 
    if st.button("Predict"): 
        result = prediction(Gender, Married, ApplicantIncome, LoanAmount, Credit_History) 
        st.success('Your loan is {}'.format(result))
        print(LoanAmount)

if __name__=='__main__': 
    main()

2021-06-23 12:07:49.878 
  command:

    streamlit run C:\Users\Dr MAXWELL ASUMENG\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [19]:
from pyngrok import ngrok
 
public_url = ngrok.connect('8501')
public_url

2021-06-23 12:07:50.452 Opening tunnel named: http-8501-c7e51903-b463-4463-b0df-6ab1eae5640a
2021-06-23 12:07:55.905 t=2021-06-23T12:07:55+0100 lvl=info msg="no configuration paths supplied"
2021-06-23 12:07:55.908 t=2021-06-23T12:07:55+0100 lvl=info msg="using configuration at default config path" path="C:\\Users\\Dr MAXWELL ASUMENG/.ngrok2/ngrok.yml"
2021-06-23 12:07:55.908 t=2021-06-23T12:07:55+0100 lvl=info msg="open config file" path="C:\\Users\\Dr MAXWELL ASUMENG\\.ngrok2\\ngrok.yml" err=nil
2021-06-23 12:07:55.926 t=2021-06-23T12:07:55+0100 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040
2021-06-23 12:07:56.889 t=2021-06-23T12:07:56+0100 lvl=info msg="tunnel session started" obj=tunnels.session
2021-06-23 12:07:56.889 t=2021-06-23T12:07:56+0100 lvl=info msg="client session established" obj=csess id=bfbfc35ba05e
2021-06-23 12:07:56.931 t=2021-06-23T12:07:56+0100 lvl=info msg=start pg=/api/tunnels id=b521d54432623802
2021-06-23 12:07:56.951 t=2021-06-23T12:07:56+01

<NgrokTunnel: "http://b0dcfdc4f0c9.ngrok.io" -> "http://localhost:8501">

2021-06-23 14:11:16.559 t=2021-06-23T14:11:16+0100 lvl=eror msg="heartbeat timeout, terminating session" obj=csess id=d7b5b2552735 clientid=19673ce33540a1864b2fc4c673969888
2021-06-23 14:11:16.607 t=2021-06-23T14:11:16+0100 lvl=eror msg="session closed, starting reconnect loop" obj=csess id=bfbfc35ba05e err="session closed"
2021-06-23 14:11:22.256 t=2021-06-23T14:11:22+0100 lvl=eror msg="failed to reconnect session" obj=csess id=bfbfc35ba05e err="x509: certificate is not valid for any names, but wanted to match tunnel.us.ngrok.com"
2021-06-23 14:11:22.917 t=2021-06-23T14:11:22+0100 lvl=eror msg="failed to reconnect session" obj=csess id=bfbfc35ba05e err="x509: certificate is not valid for any names, but wanted to match tunnel.us.ngrok.com"
